In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)
print('GPUs available: {}'.format(len(tf.config.list_physical_devices('GPU'))))

2.2.0
GPUs available: 1


In [2]:
images_dir = 'data/Images/'
out_dir = 'data/UCM_preprocessed.tfrecords'

In [72]:
def open_image(image_loc, expected_dim):
    pixel_data = np.array(Image.open(image_loc).getdata())
    
    width = np.sqrt(pixel_data.shape[0])
    channels = pixel_data.shape[1]

    if width % 1 != 0:
        print('Image {} is not square, returning empty variable'.format(image_loc))
        return None
    
    if width != expected_dim:
        print('Image {} has dimension {}, expected {}, returning empty variable'.format(image_loc, width, expected_dim))
        return None
    
    width = int(width)
    
    return pixel_data.reshape((width, width, channels)) / 255.0 #Normalised

def _int64_feature(value):
    """Wrapper for inserting int64 features into Example proto."""
    if isinstance(value, np.ndarray):
        value = value.flatten().tolist()
    elif not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def _float64_feature(value):
    """Wrapper for inserting float64 features into Example proto."""
    if isinstance(value, np.ndarray):
        value = value.flatten().tolist()
    elif not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def _bytes_feature(value):
    """Wrapper for inserting bytes features into Example proto."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def ucm_to_tfrecord_preprocessed(directory, out_file, datagen_args, expected_dim = 256, datagen_seed = 0, img_resize=None):
    if os.path.exists(out_file):
        os.remove(out_file)
    
    writer = tf.io.TFRecordWriter(out_file)
    
    label_dict = {}
    
    x = []
    y = []
    
    for label_int, label in enumerate(os.listdir(directory)):
        label_dict[label_int] = label
        for image_file in os.listdir("{}{}/".format(directory, label)):
            image_loc = "{}{}/{}".format(directory, label, image_file)
            
            image_data = open_image(image_loc, expected_dim)
                 
            if image_data is None:
                print("Image {} discarded".format(image_loc))
                continue
                
            x.append(image_data)
            y.append(label_int)
        
    shapes = {arr.shape for arr in x}
    print(shapes)
    x = np.stack(x)
    
    if not(img_resize is None):
        x = tf.image.resize(x, img_resize)
    
    y = np.stack(y)
    
    datagen = ImageDataGenerator(**datagen_args)
    print('Fitting')
    datagen.fit(x, seed = datagen_seed)
    print('Fitted')
    
    transform_iterator = datagen.flow(x, y, batch_size=1) #save_to_dir='tmp'
    
    i=0
    while i < x.shape[0]:
        xi, yi = next(transform_iterator)
        xi, yi = xi[0], yi[0]
        
        
        height = xi.shape[0]
        width = xi.shape[1]
        assert height == width

        channels = xi.shape[2]


        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(height),
            'width': _int64_feature(width),
            'channels': _int64_feature(channels),
            'label': _int64_feature(yi),
            'image_raw': _float64_feature(xi)
        }))

        writer.write(example.SerializeToString())
        
        i+=1

    with open('data/label_dict_preprocessed.pkl', 'wb') as f:
        pkl.dump(label_dict, f, pkl.HIGHEST_PROTOCOL)
            
    



In [73]:
datagen_args = dict(
    featurewise_center=True, samplewise_center=False,
    featurewise_std_normalization=False, samplewise_std_normalization=False,
    zca_whitening=True, zca_epsilon=0.1, rotation_range=0, width_shift_range=0.0,
    height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
    channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
    horizontal_flip=False, vertical_flip=False, rescale=None,
    preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None
    )

In [74]:
%%time
ucm_to_tfrecord_preprocessed(images_dir, out_dir, datagen_args, img_resize=(16,16))

Image data/Images/intersection/intersection99.tif is not square, returning empty variable
Image data/Images/intersection/intersection99.tif discarded
Image data/Images/storagetanks/storagetanks62.tif is not square, returning empty variable
Image data/Images/storagetanks/storagetanks62.tif discarded
Image data/Images/storagetanks/storagetanks60.tif is not square, returning empty variable
Image data/Images/storagetanks/storagetanks60.tif discarded
Image data/Images/storagetanks/storagetanks63.tif is not square, returning empty variable
Image data/Images/storagetanks/storagetanks63.tif discarded
Image data/Images/airplane/airplane55.tif is not square, returning empty variable
Image data/Images/airplane/airplane55.tif discarded
Image data/Images/airplane/airplane59.tif is not square, returning empty variable
Image data/Images/airplane/airplane59.tif discarded
Image data/Images/airplane/airplane76.tif is not square, returning empty variable
Image data/Images/airplane/airplane76.tif discarde